## 07 - Geographical Formatting
Here is pathway toward aggregating, grouping and narrowing down your dataframe. (There is no right way--there are just ways that work better than others.)

Import the data into pandas

In [477]:
import numpy as np
import pandas as pd
from datetime import datetime
df = pd.read_csv("/Users/laurabejderjensen/Desktop/GitHub/federal_judges_appointment/data/data.csv")

# I drop the rows where the judge was confirmed by voice call.
df = df[df['record_vote_number'].notna()]

And change the `confirmation_date` column from string to datetime format. 

In [478]:
df['confirmation_date'] = pd.to_datetime(df.confirmation_date)

In [479]:
df['yea_votes'] = df['yea_votes'].astype(int)
df['nay_votes'] = df['nay_votes'].astype(int)

Order the dataframe by the confirmation date. This will secure that the final putput will present the judges cronologically (with the lastest confirmation first).

In [480]:
#df1 = df.groupby('circuit').order_by['name'].value_counts().reset_index(name='count')
df1 = df.sort_values('confirmation_date', ascending=False)

Here I build the **"article:"** field of my geojson doc. This article contains all the information about each judge in readable text (and tables).

In [481]:
#Moving the values from the two right columns into a new column
#One that human readers can understand
df1["string"] = "<strong style=\"font-size:18px;\">" + df["name"] + "</strong> <br><br>" + df["name"] + " was confirmed to be a United States \
    circuit judge of the United States Court of Appeals for the " + df['circuit'] + " Circuit by the senate on the " + df1["confirmation_date"].astype(str) + ".<br>\
    " + df1["yea_votes"].astype(str) + " senators voted yea to confirm\
    while " + df1["nay_votes"].astype(str) + " voted nay. \
    <table>\
        <col width=\"100\"> \
            <col width=\"70\"> \
                <col width=\"70\"> \
                    <col width=\"70\"> \
                        <tr> \
                            <th style=\"text-align:left\">Party votes</th> \
                            <th style=\"text-align:right\">Yea</th> \
                            <th style=\"text-align:right\">Nay</th> \
                            <th style=\"text-align:right\">No vote</th> \
                        </tr>\
                        <tr>\
                            <td>Democrats</td>\
                            <td style=\"text-align:right\">" + df1["D_yea"].astype(str) + "</td>\
                            <td style=\"text-align:right\">" + df1["D_nay"].astype(str) + "</td>\
                            <td style=\"text-align:right\">" + df1["D_no_vote"].astype(str) + "</td>\
                        </tr>\
                        <tr>\
                            <td>Republicans</td>\
                            <td style=\"text-align:right\">" + df1["R_yea"].astype(str) + "</td> \
                            <td style=\"text-align:right\">" + df1["R_nay"].astype(str) + "</td> \
                            <td style=\"text-align:right\">" + df1["R_no_vote"].astype(str) + "</td> \
                        </tr> \
                        <tr> \
                            <td>Independents</td> \
                            <td style=\"text-align:right\">" + df1["I_yea"].astype(str) + "</td> \
                            <td style=\"text-align:right\">" + df1["I_nay"].astype(str) + "</td> \
                            <td style=\"text-align:right\">" + df1["I_no_vote"].astype(str) + "</td> \
                        </tr> \
    </table>"

Then I merge all the articles into **one row per each United States circuit court of appeals**. 

In [482]:
#this is just two different ways to do this:
#one using % as the wild card
#and one using .format and {0} as the wild card

output = df1.groupby('circuit')['string'].apply(lambda x: "<div class='confirmed_judges'><h2><b>Confirmed Circuit Judges</b></h2><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')


I save the `circuit` column as the `properties.name` column too.

In [483]:
#Add some color
#output['properties.color'] = "#35476E"
output['properties.name'] = output['circuit']

### geojson > pandas > mapbox

In this section I prepare the geojson file and merge it with the judge data.

In [484]:
#Some nice imports
import requests
import json
import numpy as np
import pandas as pd
from pandas import json_normalize


Load the geojson file Exported from Mapshaper

In [485]:
with open('map/courts13_join.json') as json_data:
    geometry_data = json.load(json_data)

In [486]:
##Normalize the hierarchy  so you have simple rows in a dataframe
##Note that you need to extract it from geometry_data['features']
geo = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')

Now I begin **building the properties** necessary for the geojson file.

**name:** This is the name of the location (the circuit) that shows up when you rollover the shape.

(*lambda* Which you will see a lot of below, Is what is called an anonymous function or one-line function. It allows you to do transformations on iterated values, along with other stuff...Here it changes the line to title case)

But first I loop through all the districts and create the name in letters not numbers. It looks better. Afterwards I add the rest of the name, so it says "This is the x Circuit".

In [487]:
names = []
for index, row in geo.iterrows():
    name = {}
    name['properties.District_N'] = row['properties.District_N']
    if row['properties.District_N'] == '1':
        name['properties.name'] = "First"
    elif row['properties.District_N'] == '2':
        name['properties.name'] = "Second"
    elif row['properties.District_N'] == '3':
        name['properties.name'] = "Third"
    elif row['properties.District_N'] == '4':
        name['properties.name'] = "Fourth"
    elif row['properties.District_N'] == '5':
        name['properties.name'] = "Fifth"
    elif row['properties.District_N'] == '6':
        name['properties.name'] = "Sixth"
    elif row['properties.District_N'] == '7':
        name['properties.name'] = "Seventh"
    elif row['properties.District_N'] == '8':
        name['properties.name'] = "Eighth"
    elif row['properties.District_N'] == '9':
        name['properties.name'] = "Ninth"
    elif row['properties.District_N'] == '10':
        name['properties.name'] = "Tenth"
    elif row['properties.District_N'] == '11':
        name['properties.name'] = "Eleventh"
    elif row['properties.District_N'] == 'District of Columbia':
        name['properties.name'] = "District of Columbia"
    else:
        pass
    names.append(name)
names

geo1 = pd.DataFrame(names)

In [488]:
geography = geo.merge(geo1, how='left', on='properties.District_N')

In [489]:
def add_text(cell): 
    return "This is the " + cell + " Circuit"


**headline:** This is the lead sentence or bullet point displayed when you rollover the shape.


In [490]:
#df2['properties.headline'] = df2['properties.District_N'].apply(lambda x: "This is in district " + x)
geography['properties.headline'] = geography['properties.name'].apply(add_text)

In [491]:
data = geography.merge(output, how='left', on='properties.name')

In [492]:
data = data.drop('circuit', axis=1)


**color:** This will set the color for every shape. Here we are making semi-random colors for every single shape...Not a good thing to do. But definitely have a lot of funWith your color algorithms here. You want the colors to Reflect different ranges of values. Random is the last thing you want to do. But the function below builds random hexadecimal color values.



**group_id:** This separates different groups with the pulldown menu. Showing everything should be groups 0, Individual groups should begin at 1 And go up in order (2, 3, 4). If you want to have completely different groupsShowing different information but in the same place, talk to me about that.

**group_name:** should correspond to **group_id:** It is the name that shows up in the menubar.

In [493]:
data['properties.group_id'] = 0

## Creating a new dataframe with information from each congress
Because I want the information in the map to be grouped by congress number I create an additional dataframe with all the same information but this time with group-ids corresponding to the congress number.
- 107th congress = 1
- 108th congress = 2
- 109th congress = 3 <br>
...
- 117th congress = 11

In [494]:
congresses_df = df1.groupby(['congress_no', 'circuit'])['string'].apply(lambda x: "<div class='confirmed_judges'><h2><b>Confirmed Circuit Judges</b></h2><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')

In [495]:
numbers = []
for index, row in congresses_df.iterrows():
    number = {}
    if row.congress_no == 107:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '1'
    elif row.congress_no == 108:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '2'
    elif row.congress_no == 109:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '3'
    elif row.congress_no == 110:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '4'
    elif row.congress_no == 111:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '5'
    elif row.congress_no == 112:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '6'
    elif row.congress_no == 113:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '7'
    elif row.congress_no == 114:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '8'
    elif row.congress_no == 115:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '9'
    elif row.congress_no == 116:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '10'
    else:
        number['congress_no1'] = row.congress_no
        number['properties.group_id'] = '11'

    numbers.append(number)
names

df_numbers = pd.DataFrame(numbers)

In [496]:
# Assign the properties.group:id to the rows.
congresses = pd.concat([congresses_df, df_numbers], axis = 1)
congresses = congresses.drop(['congress_no1'], axis = 1)

In [497]:
congresses['properties.name'] = congresses['circuit']
congresses = congresses.drop('circuit', axis=1)

In [498]:
data1 = congresses.merge(geography, how='right', on='properties.name')
data1 = data1[['type', 'properties.District_N', 'properties.District_Name', 'geometry.coordinates', 'geometry.type', 'properties.name', 'properties.headline', 'properties.article', 'properties.group_id']]

In [499]:
all_data = pd.concat([data, data1], ignore_index=True, axis=0)

In [500]:
def category_num(cell):
    cate_list = ['All congresses',
                 '107th Congress',
                 '108th Congress',
                 '109th Congress',
                 '110th Congress',
                 '111th Congress',
                 '112th Congress',
                 '113th Congress',
                 '114th Congress',
                 '115th Congress',
                 '116th Congress',
                 '117th Congress'
                 ]  
    congress = int(cell)
    for i in range(0,12):
        if congress == i:
            return str(cate_list[i])

In [501]:
all_data['properties.group_name'] = all_data['properties.group_id'].apply(category_num)

## Setting the colors
Here I set the colors for each circuit.

In [502]:
color = ["#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a", "#ffff99", "#b15928", "#a6cee3"]

def colors(cell):
    if cell == 'District of Columbia':
        return color[0]
    else:
        return color[int(cell)]

In [503]:
all_data['properties.color'] = all_data['properties.District_N'].apply(colors)

In [504]:
all_data.loc[all_data['properties.group_name'] == "All congresses"]

,type,properties.District_N,properties.District_Name,geometry.coordinates,geometry.type,properties.name,properties.headline,properties.article,properties.group_id,properties.group_name,properties.color
0,Feature,11,United States Court of Appeals for the Elevent...,"[[[[-87.987045288, 35.007518768], [-86.8366699...",MultiPolygon,Eleventh,This is the Eleventh Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#a6cee3
1,Feature,9,United States Court of Appeals for the Ninth C...,"[[[[-109.044883728, 36.998630524], [-109.04572...",MultiPolygon,Ninth,This is the Ninth Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#ffff99
2,Feature,8,United States Court of Appeals for the Eighth ...,"[[[[-89.716941833, 36.00151825], [-89.72039032...",MultiPolygon,Eighth,This is the Eighth Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#6a3d9a
3,Feature,10,United States Court of Appeals for the Tenth C...,"[[[-104.052841187, 41.00169754], [-102.9998245...",Polygon,Tenth,This is the Tenth Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#b15928
4,Feature,2,United States Court of Appeals for the Second ...,"[[[[-71.964263916, 41.340965271], [-71.9668426...",MultiPolygon,Second,This is the Second Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#33a02c
5,Feature,4,United States Court of Appeals for the Fourth ...,"[[[[-75.541755676, 39.450660706], [-75.5586166...",MultiPolygon,Fourth,This is the Fourth Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#e31a1c
6,Feature,District of Columbia,United States Court of Appeals for the Distric...,"[[[-77.026161194, 38.801475525], [-77.02017974...",Polygon,District of Columbia,This is the District of Columbia Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#1f78b4
7,Feature,7,United States Court of Appeals for the Seventh...,"[[[[-90.237174988, 41.684024811], [-90.2111663...",MultiPolygon,Seventh,This is the Seventh Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#cab2d6
8,Feature,6,United States Court of Appeals for the Sixth C...,"[[[[-82.592788696, 38.41859436], [-82.59582519...",MultiPolygon,Sixth,This is the Sixth Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#ff7f00
9,Feature,5,United States Court of Appeals for the Fifth C...,"[[[[-90.893592834, 29.046777725], [-90.8644332...",MultiPolygon,Fifth,This is the Fifth Circuit,<div class='confirmed_judges'><h2><b>Confirmed...,0,All congresses,#fdbf6f


And change the title of each article, so it includes the name of the circuit.

In [505]:
def add_circuit_name(row):
    return row['properties.article'].replace('Confirmed Circuit Judges', 'Confirmed Circuit Judges for the ' + row['properties.name'] + ' Circuit')
    
all_data['properties.article'] = all_data.apply(add_circuit_name, axis=1)

At last I turn the special properties and the geographical information back into **json format** we orient by records because that gives us an array of dictionaries.

In [506]:
ok_json = json.loads(all_data.to_json(orient='records'))

But because we had to normalize the hierarchy of the geojson document we now have to rebuild the hierarchy so this json document becomes geojson, the function below does just that:


In [507]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [508]:
geo_format = process_to_geojson(ok_json)

Now we can export this to a file!

In [509]:
#Variable name
with open('map/geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('map/geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)